old_なEATを処理する (一度だけ実行)

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import librosa
import soundfile as sf

#dataを作成するプログラム群
#中村さんのデータはA,Bなどのフォルダ構成を想定(食行動DBの研究フォルダからダウンロード、展開したもの)
#現状、食行動DBのdatasetは強ラベル(正確なもの)がついている。

#実験用のデータフォルダ
output_folder = "/scratch/acf15294oh/database/test_9_6"




# 中村さんのデータフォルダ
old_data_folder = "/scratch/acf15294oh/database/behavior_old"


#ファイル階層は以下を想定
#
# EAT_old(data_folder)
#    |
#    |--A
#    |  |--001_CBG_22k_MAN01_1.wav
#    |   --001_CBG_MAN01_01.txt
#     --B
#       |--001_CBG_22k_MHT01_1.wav
#        --001_CBG_MHT01_01.txt

wav_output_folder = old_data_folder + "/wav_div"
text_output_folder = old_data_folder + "/text_strong_div"

# 利用したいマイクのリスト
Mics = {
    #"SM": 1,
     "TM01": 2,
    # "TM02": 3,
    # "EM01": 4,
    # "EM02": 5,
    #"CM": 6
}

Augmentation = [
    #"ngram", 未実装
    "label_start_cut",
    #"enhancement"
]

#切り分ける音声ファイルの長さの最小値と最大値を設定
segment = 10
max_segment_sec = 15


old_wav_folder = output_folder + "/old_wav_ful"
old_txt_folder = output_folder + "/old_txt_ful"
old_div_folder = output_folder + "/old_wav_aug"
old_div_text_folder = output_folder + "/old_text_aug"


## ***中村さんのデータを取り出し、各マイクフォルダへと割り当てる

In [2]:



# 各フォルダに対して処理を行う
for folder in Mics:
    # 完全なパスを作成
    folder_path = os.path.join(old_wav_folder, folder)
    
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        #print(f"Created folder: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")

if not os.path.exists(old_txt_folder):
    os.makedirs(old_txt_folder)
    #print(f"Created folder: {old_txt_folder}")
else:
    print(f"Folder already exists: {old_txt_folder}")

import shutil

def move_text_files(source_folder, destination_folder):
    # source_folder の中を再帰的に探索
    for dirpath, dirnames, filenames in os.walk(source_folder):
        for filename in filenames:
            if not filename.startswith('._') and filename.endswith('.txt'):  # テキストファイルをチェック
                source_file = os.path.join(dirpath, filename)
                filename = filename.replace("001_", "")
                destination_file = os.path.join(destination_folder, filename)


                # ファイルを移動する前に、同名のファイルが目的地に存在するか確認
                if os.path.exists(destination_file):
                    print(f"Error: {destination_file} already exists.")
                else:
                    shutil.copy(source_file, destination_file)  # ファイルを移動
                    #print(f"copy: {source_file} -> {destination_file}")

def move_wav_files(source_folder, destination_folder):
    # source_folder の中を再帰的に探索
    for dirpath, dirnames, filenames in os.walk(source_folder):
        print(dirpath)
        for filename in tqdm(filenames):
            if not filename.startswith('._') and filename.endswith('.wav'):  # テキストファイルをチェック
                for mic in Mics:
                    source_file = os.path.join(dirpath, filename)
                    #filename = filename.replace("001_", "")
                    mic_num = source_file.split("_")[-1].split(".")[0]
                    if str(Mics[mic]) == mic_num:
                        filename = mic + "/" +filename
                        destination_file = os.path.join(destination_folder, filename)
                        #print(destination_file)

                        # ファイルを移動する前に、同名のファイルが目的地に存在するか確認
                        if os.path.exists(destination_file):
                            print(f"Error: {destination_file} already exists.")
                        else:
                            shutil.copy(source_file, destination_file)  # ファイルを移動
                            #print(f"copy: {source_file} -> {destination_file}")


def resample_audio_files(source_folder, target_folder, target_sr=16000):
    # source_folder の中を再帰的に探索
    for dirpath, dirnames, filenames in os.walk(source_folder):
        print(dirpath)
        for filename in tqdm(filenames):
            if filename.lower().endswith(('.wav', '.flac', '.mp3')):  # 対応する音声ファイル形式
                source_file = os.path.join(dirpath, filename)


                filename = filename.replace("_22k_", "_")
                filename = filename.replace("001_", "")
                target_file = os.path.join(dirpath, filename)

                
                # 音声を読み込み、サンプリングレートを変更
                audio, sr = librosa.load(source_file, sr=None)  # 元のサンプリングレートで読み込み
                if sr != target_sr:
                    audio_resampled = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
                    # レート変更後の音声を保存
                    sf.write(target_file, audio_resampled, target_sr)
                    #print(f"Resampled and saved {source_file} -> {target_file} at {target_sr} Hz")
                    if os.path.exists(source_file):
                        # ファイルを削除
                        os.remove(source_file)
                        #print(f"Deleted: {source_file}")
        
                else:
                    # サンプリングレートが既に目的のものであれば、何もしない
                    #print(f"nothing to do {source_file} -> {target_file}")
                    continue






source = old_data_folder
destination = old_txt_folder
print("move text files!")
move_text_files(source, destination)
print("finish text files!")

source = old_data_folder
destination = old_wav_folder
print("move wav files!")
move_wav_files(source, destination)
print("finish wav files!")

source = old_wav_folder
destination = old_wav_folder
print("resample audio_files!")
resample_audio_files(source, destination)
print("all finished!")

move text files!
finish text files!
move wav files!
/scratch/acf15294oh/database/behavior_old


0it [00:00, ?it/s]


/scratch/acf15294oh/database/behavior_old/C


100%|██████████| 185/185 [00:00<00:00, 216.17it/s]


/scratch/acf15294oh/database/behavior_old/B


...

finish wav files!
resample audio_files!
/scratch/acf15294oh/database/behavior_TM01/old_wav_ful


0it [00:00, ?it/s]


/scratch/acf15294oh/database/behavior_TM01/old_wav_ful/TM01


100%|██████████| 148/148 [00:26<00:00,  5.50it/s]

all finished!


# **データフォルダからテキスト情報を収集、DFに格納**

In [3]:
import os
import pandas as pd
import re

text_folder = old_txt_folder

# フォルダ内の全てのテキストファイルのリストを取得
files = [f for f in os.listdir(text_folder) if f.endswith('.txt')]

# 全ファイルのデータを保持するリスト
all_rows = []

# 各ファイルに対して処理を行う
for file_name in files:
    file_path = os.path.join(text_folder, file_name)
    speaker = file_name.split("_")[-2]
    food = file_name.split("_")[0]
    
    if len(file_name.split("_")) == 4:
        index = file_name.split("_")[1]
        food = food + "_" + index
    
    # ファイルを開いて各行をリストとして読み込む
    with open(file_path, 'r', encoding='utf-8') as file:
         rows = [
                    [float(row[0])] + [float(row[1])] + row[2:]
                    for row in (
                        [item for item in re.split(r'\s+', line.strip()) if item]  # 連続する空白で分割し、空の要素を除外
                        for line in file
                    )
                ]
    
    # 読み込んだ各行に話者名を追加し、全体のリストに追加
    for row in rows:
        all_rows.append([speaker] + [food] + row)

# 全データを含むデータフレームを作成
df = pd.DataFrame(all_rows, columns=['Speaker', "Food", 'StartTime','EndTime', 'Behavior']) 

# データフレームを表示

print(f"総行数: {len(df)}")
print(df)  # データフレームの最初の数行を表示

総行数: 24193
      Speaker Food   StartTime     EndTime Behavior
0       MKG01  GUM    2.207962    2.628346    right
1       MKG01  GUM    2.796878    3.516454    right
2       MKG01  GUM    4.270115    4.584456    right
3       MKG01  GUM    5.177160    5.529373    right
4       MKG01  GUM    5.828565    6.148587    right
...       ...  ...         ...         ...      ...
24188   MTG01  CBG  366.286825  366.559630     left
24189   MTG01  CBG  366.960815  367.205815     left
24190   MTG01  CBG  367.575964  367.820964     left
24191   MTG01  CBG  371.015453  371.715453  swallow
24192   MTG01  CBG  378.359804  379.059804  swallow

[24193 rows x 5 columns]


In [4]:
#複数回実行するとエラーを吐くので最初から実行し直して！！

speaker_list = df['Speaker'].unique().tolist()
rows = []

for i in range(len(speaker_list)):

    foods = df[df.Speaker== speaker_list[i]].Food.unique().tolist()
    for food in foods:
        start = df[(df.Speaker == speaker_list[i]) & (df.Food == food)].StartTime.min()
        end = df[(df.Speaker==speaker_list[i]) & (df.Food == food)].EndTime.max()
        
        right_len = len(df[(df.Speaker==speaker_list[i]) & (df.Behavior=="right") & (df.Food == food)])
        left_len = len(df[(df.Speaker==speaker_list[i]) & (df.Behavior=="left") & (df.Food == food)])
        chew_len = right_len + left_len
        swallow_len = len(df[(df.Speaker==speaker_list[i]) & (df.Behavior=="swallow") & (df.Food == food)])
        #all_data_len = len(df[df.Speaker==speaker_list[i]])
    
        rows.append([speaker_list[i], food, left_len, right_len, chew_len, swallow_len, start, end])

df_count = pd.DataFrame(rows, columns=['Speaker', "Food", 'Left', 'Right', 'Chewing', 'Swallowing', 'StartTime', 'EndTime']) 
print(df_count)

    Speaker   Food  Left  Right  Chewing  Swallowing  StartTime     EndTime
0     MKG01    GUM   109    113      222           7   2.207962  180.986976
1     MKG01    CBG   295    400      695          28   0.437256  432.355251
2     MKG01  W20_1     0      0        0          12   3.083867   80.779070
3     MKG01  W20_2     0      0        0          10   3.079702   70.204555
4     MKG01  RTZ_2    94    172      266          16   1.285022  229.598659
..      ...    ...   ...    ...      ...         ...        ...         ...
143   MMK01  W20_2     0      0        0           9   3.253525   75.426289
144   MMK01    GUM    90    102      192           2   2.880862  178.690703
145   MMK01  RTZ_1   107     47      154           6   2.832851  165.465911
146   MMK01  W20_1     0      0        0          10   6.200624   84.405442
147   MDN01    CBG   206    496      702          23   0.837725  551.046698

[148 rows x 8 columns]


In [5]:
import os

if "enhancement" in Augmentation:
    folder = "conbined"
    folder_path = os.path.join(old_div_folder, folder)
    
    # ディレクトリが存在しない場合は作成
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
    else:
        print(f"Folder already exists: {folder_path}")

else:
    # 各フォルダに対して処理を行う
    for folder in Mics:
        # 完全なパスを作成
        folder_path = os.path.join(old_div_folder, folder)
        
        # ディレクトリが存在しない場合は作成
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Created folder: {folder_path}")
        else:
            print(f"Folder already exists: {folder_path}")

if not os.path.exists(old_div_text_folder):
        os.makedirs(old_div_text_folder)
        print(f"Created folder: {old_div_text_folder}")
else:
    print(f"Folder already exists: {old_div_text_folder}")


Created folder: /scratch/acf15294oh/database/behavior_TM01/old_wav_aug/TM01
Created folder: /scratch/acf15294oh/database/behavior_TM01/old_text_aug


In [6]:
import wave
import numpy as np
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import soundfile as sf
from tqdm import tqdm 

fr = 16000
max_duration = segment

speaker_list = df['Speaker'].unique().tolist()



for Speaker in tqdm(speaker_list):

    foods = df[df.Speaker== Speaker].Food.unique().tolist()
    for Food in foods: 
        sp_df = df[(df.Speaker==Speaker) & (df.Food==Food)]


        if "label_start_cut" in Augmentation:
            for ite in range(len(sp_df)-1):
                behavior = []
                behavior_time = []
                start_time = sp_df.iloc[ite].StartTime
                end_time = sp_df.iloc[ite].EndTime
                duration = 0

                behavior.append(sp_df.iloc[ite].Behavior)
                behavior_time.append([start_time, end_time])
                ite += 1

                while ite < len(sp_df):
                    
            
                    next_end_time = sp_df.iloc[ite].EndTime 
                    duration = next_end_time - start_time
            
                    if (duration<max_duration):            
                        behavior.append(sp_df.iloc[ite].Behavior)
                        next_start_time = sp_df.iloc[ite].StartTime 
                        behavior_time.append([next_start_time, next_end_time])
                        end_time = next_end_time
                    else:
                        break
                    ite += 1

                start_time = start_time - 0.1
                end_time = end_time + 0.1
                duration = end_time-start_time

                short_start_time = round(start_time, 2)
                integer = str(short_start_time).split(".")[0].rjust(4,"0")
                decimal = str(short_start_time).split(".")[1].ljust(2,"0")
                #print(duration)
            
                
                
                if "enhancement" in Augmentation:
                    # 各マイクのファイルパスをリストで指定
                    mic_files = [
                        f"{old_wav_folder}/{mic}/{Food}_{Speaker}_{Mics[mic]}.wav"
                        for mic in Mics
                    ]

                    # 最初のオーディオファイルを読み込み
                    y_combined, sr = librosa.load(mic_files[0], sr=fr, offset=start_time, duration=duration)

                    # 残りのオーディオファイルを読み込み、加算
                    for mic_file in mic_files[1:]:
                        y, _ = librosa.load(mic_file, sr=fr, offset=start_time, duration=duration)
                        y_combined += y

                    # 加算された信号を保存
                    wav_file = f"{old_div_folder}/{folder}/eat_{Speaker}_{folder}_{Food}_{integer}_{decimal}.wav"
                    sf.write(wav_file, y_combined, sr)
                    
                else:    
                    for mic in Mics:
                        data_file = old_wav_folder + f"/{mic}/{Food}_{Speaker}_{Mics[mic]}.wav"
                        #print(data_file)
                        y, sr = librosa.load(data_file, sr=fr, offset=start_time, duration=duration) #オーディオファイルの読み込み
                        wav_file = old_div_folder + f"/{folder}/eat_{Speaker}_{folder}_{Food}_{integer}_{decimal}.wav"
                        sf.write(wav_file, y, sr) 
                        #print(wav_file)
                
                text_file = old_div_text_folder + f"/eat_{Speaker}_{Food}_{integer}_{decimal}.txt"


                
                with open(text_file, "w") as txt_file:
                    for label, times in zip(behavior,behavior_time):
                        beh_start_time = times[0] - start_time  # 時間差を計算
                        beh_end_time = times[1] - start_time  # 時間差を計算
                        
                        if label in ["right", "left", "front"]:
                            txt_file.write(f"{beh_start_time}\t{beh_end_time}\tch\n")
                        elif label == "swallow":
                            txt_file.write(f"{beh_start_time}\t{beh_end_time}\tsw\n")
                        elif label in ["noise", "other"]:
                            txt_file.write(f"{beh_start_time}\t{beh_end_time}\tno\n")
                        else:
                            print("エラー：不正な文字列が検出されました")
                            print(label)

        else:
             ite = 0
             while ite < len(sp_df):
                behavior = []
                behavior_time = []
                start_time = sp_df.iloc[ite].StartTime
                end_time = sp_df.iloc[ite].EndTime
                duration = 0
                
                behavior.append(sp_df.iloc[ite].Behavior)
                behavior_time.append([start_time, end_time])
                ite += 1
            
                # sp_dfの範囲を超えないようにチェック
                while ite < len(sp_df):
                    next_start_time = sp_df.iloc[ite].StartTime 
                    next_end_time = sp_df.iloc[ite].EndTime 
                    duration = next_end_time - start_time
            
                    if (duration<max_duration):            
                        behavior.append(sp_df.iloc[ite].Behavior)
                        behavior_time.append([next_start_time, next_end_time])
                        end_time = next_end_time
                    else:
                        break
                    ite += 1
            
                start_time = start_time
                end_time = end_time
                duration = end_time-start_time

                short_start_time = round(start_time, 2)
                integer = str(short_start_time).split(".")[0].rjust(4,"0")
                decimal = str(short_start_time).split(".")[1].ljust(2,"0")
                #print(duration)
            
                
                if "enhancement" in Augmentation:
                    # 各マイクのファイルパスをリストで指定
                    mic_files = [
                        f"{old_wav_folder}/{mic}/{Food}_{Speaker}_{Mics[mic]}.wav"
                        for mic in Mics
                    ]

                    # 最初のオーディオファイルを読み込み
                    y_combined, sr = librosa.load(mic_files[0], sr=fr, offset=start_time, duration=duration)

                    # 残りのオーディオファイルを読み込み、加算
                    for mic_file in mic_files[1:]:
                        y, _ = librosa.load(mic_file, sr=fr, offset=start_time, duration=duration)
                        y_combined += y

                    # 加算された信号を保存
                    wav_file = f"{old_div_folder}/{folder}/eat_{Speaker}_{folder}_{Food}_{integer}_{decimal}.wav"
                    sf.write(wav_file, y_combined, sr)
                    
                else:    
                    for mic in Mics:
                        data_file = old_wav_folder + f"/{mic}/{Food}_{Speaker}_{Mics[mic]}.wav"
                        #print(data_file)
                        y, sr = librosa.load(data_file, sr=fr, offset=start_time, duration=duration) #オーディオファイルの読み込み
                        wav_file = old_div_folder + f"/{folder}/eat_{Speaker}_{folder}_{Food}_{integer}_{decimal}.wav"
                        sf.write(wav_file, y, sr) 
                        #print(wav_file)
                
                text_file = old_div_text_folder + f"/eat_{Speaker}_{Food}_{integer}_{decimal}.txt"


                
                with open(text_file, "w") as txt_file:
                    for label, times in zip(behavior,behavior_time):
                        beh_start_time = times[0] - start_time  # 時間差を計算
                        beh_end_time = times[1] - start_time  # 時間差を計算
                        
                        if label in ["right", "left", "front"]:
                            txt_file.write(f"{beh_start_time}\t{beh_end_time}\tch\n")
                        elif label == "swallow":
                            txt_file.write(f"{beh_start_time}\t{beh_end_time}\tsw\n")
                        elif label in ["noise", "other"]:
                            txt_file.write(f"{beh_start_time}\t{beh_end_time}\tno\n")
                        else:
                            print("エラー：不正な文字列が検出されました")
                            print(label)


100%|██████████| 32/32 [00:25<00:00,  1.23it/s]


In [7]:
import os
import wave

def list_wav_longer_than_20s(folder_path):
    long_durations = []
    long_files_full_path = []  # フルパスを格納する配列

    # フォルダ内のすべてのファイルを再帰的にリストアップ
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # ファイルがWAV形式であるか確認
            if file.endswith('.wav'):
                # WAVファイルのフルパスを取得
                file_path = os.path.join(root, file)
                
                # WAVファイルを開く
                try:
                    with wave.open(file_path, 'r') as wav_file:
                        # フレーム数を取得
                        frames = wav_file.getnframes()
                        # レート（1秒あたりのフレーム数）を取得
                        rate = wav_file.getframerate()
                        # 持続時間を計算
                        duration = frames / float(rate)
                        
                        # 持続時間が20秒以上の場合にリストに追加
                        if duration > 17:
                            long_durations.append((file_path, duration))  # ファイルのフルパスを保存
                except wave.Error as e:
                    print(f"Error opening {file_path}: {e}")

    # 持続時間でリストを降順に並び替え
    long_durations.sort(key=lambda x: x[1], reverse=True)
    
    # 並び替えた結果を出力し、フルパスを配列に格納
    for file_path, duration in long_durations:
        long_files_full_path.append(file_path)
        print(f"{file_path}: {duration}秒")

    return long_files_full_path

# 使用例
folder_path = wav_output_folder
long_files = list_wav_longer_than_20s(folder_path)
"""
# 出力されたファイルパスを使ってファイルを削除
for file_path in long_files:
    print(f"Deleting {file_path}...")
    os.remove(file_path)
    print(f"Deleted {file_path}")
    """

print("All specified files have been deleted.")


All specified files have been deleted.


In [8]:
long_files

[]

In [9]:
import os
import shutil
from pathlib import Path
import numpy as np
from scipy import signal
from scipy.io.wavfile import read, write

# 設定
folder_path = old_div_folder  # 検索するフォルダのパス
cutoff = 100.0  # カットオフ周波数
numtaps = 1023  # FIRフィルタの長さ（奇数が推奨）

def high_pass_filter(input_path: Path, output_path: Path, cutoff: float, numtaps: int) -> None:
    """入力音声に対してHigh-Pass Filter（HPF）を適用し、結果を別のファイルに保存する。

    Args:
        input_path (Path): 入力音声のパス。
        output_path (Path): 出力音声の保存先パス。
        cutoff (float): カットオフ周波数。
        numtaps (int): FIRフィルタのタップ数（フィルタの長さ）。
    """
    fs, data = read(str(input_path))
    # データを-1.0から1.0の範囲に正規化
    data = data.astype(np.float32)
    max_int16 = 2**15
    data /= max_int16

    # 高域通過フィルタの設計
    hpf = signal.firwin(numtaps, cutoff, pass_zero=False, fs=fs)
    
    # フィルタを適用
    filtered_data = signal.lfilter(hpf, 1.0, data)
    
    # データをint16に戻す
    filtered_data = np.clip(filtered_data, -1, 1)
    filtered_data = (filtered_data * max_int16).astype(np.int16)
    
    write(str(output_path), fs, filtered_data)

def apply_filter_to_files(folder_path: str, cutoff: float, numtaps: int) -> None:
    """指定したフォルダ内のすべてのWAVファイルにHPFを適用し、元のファイルを上書きする。

    Args:
        folder_path (str): WAVファイルが存在するフォルダのパス。
        cutoff (float): カットオフ周波数。
        numtaps (int): フィルタのタップ数。
    """
    for root, dirs, files in os.walk(folder_path):
        for file in tqdm(files):
            if file.endswith(".wav"):
                full_path = Path(root) / file
                temp_path = Path(root) / f"temp_{file}"
                #print(f"Processing: {full_path}")
                
                # フィルタリングを実施
                high_pass_filter(full_path, temp_path, cutoff, numtaps)
                
                # 元のファイルを削除して、新しいファイル名を元のファイル名に変更
                os.remove(full_path)
                shutil.move(temp_path, full_path)
                #print(f"Updated: {full_path}")

# 実行
apply_filter_to_files(folder_path, cutoff, numtaps)


0it [00:00, ?it/s]
  0%|          | 0/2350 [00:00<?, ?it/s]

100%|██████████| 2350/2350 [01:01<00:00, 38.39it/s]


## 余分なフォルダの削除

In [10]:


def delete_folder_recursively(folder_path):
    # フォルダが存在するかを確認
    if os.path.exists(folder_path):
        # フォルダを再帰的に削除
        shutil.rmtree(folder_path)
        print(f"フォルダ '{folder_path}' を再帰的に削除しました。")
    else:
        print(f"指定されたフォルダ '{folder_path}' は存在しません。")

delete_folder_recursively(old_wav_folder)
delete_folder_recursively(old_txt_folder)


フォルダ '/scratch/acf15294oh/database/behavior_TM01/old_wav_ful' を再帰的に削除しました。
フォルダ '/scratch/acf15294oh/database/behavior_TM01/old_txt_ful' を再帰的に削除しました。
